#  Преобразование текстов в последовательность индексов токенов. Torchtext.

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/text/stable/
* https://pytorch.org/text/stable/vocab.html
* https://pytorch.org/text/stable/transforms.html

## Задачи для совместного разбора

1\. Рассмотрите основные шаги по преобразованию текста в последовательность индексов токенов.

In [1]:
corpus = [
    "Студенты усердно занимаются стремясь получить знания и достичь успеха",
    "Студенты активно участвуют в общественной жизни университета"
]

In [2]:
from nltk import word_tokenize
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
corpus_t = [
    word_tokenize(text.lower())
    for text in corpus
]
corpus_t

[['студенты',
  'усердно',
  'занимаются',
  'стремясь',
  'получить',
  'знания',
  'и',
  'достичь',
  'успеха'],
 ['студенты',
  'активно',
  'участвуют',
  'в',
  'общественной',
  'жизни',
  'университета']]

In [4]:
words = set()
words.update(corpus_t[0])
words.update(corpus_t[1])
words = list(words)

In [5]:
itos = words

In [6]:
stoi = {v: idx for idx, v in enumerate(itos)}

In [7]:
itos[0], stoi['и']

('успеха', 12)

In [8]:
corpus_i = [
    [stoi[i] for i in tokens]
    for tokens in corpus_t
]

In [9]:
corpus_i

[[2, 13, 14, 6, 10, 11, 12, 9, 0], [2, 3, 7, 8, 5, 4, 1]]

In [10]:
corpus_i[1].extend([-1, -1])

In [11]:
import torch as th

In [12]:
th.tensor(corpus_i) # padding

tensor([[ 2, 13, 14,  6, 10, 11, 12,  9,  0],
        [ 2,  3,  7,  8,  5,  4,  1, -1, -1]])

2\. Рассмотрите процесс создания `Vocab` из `torchtext`.

In [13]:
import torchtext

In [14]:
from torchtext.vocab import build_vocab_from_iterator

In [15]:
vocab = build_vocab_from_iterator(corpus_t)

In [16]:
vocab.lookup_indices(['занимаются', 'университета'])

[5, 11]

3\. Примените преобразование `AddToken` из пакета `torchtext`.

In [17]:
from torchtext.transforms import AddToken

In [18]:
add = AddToken(token=999, begin=False)
add(corpus_i)

[[2, 13, 14, 6, 10, 11, 12, 9, 0, 999], [2, 3, 7, 8, 5, 4, 1, -1, -1, 999]]

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Опишите класс `Vocab`. При создании объекта `Vocab` в конструктор передается набор текстов, предварительно разбитых на токены. Объект должен позволять:
* по токену получить его уникальный индекс (в случае отсутствия токена в словаре вернуть 1)
* по индексу токена получить сам токен (в случае отсутствия токена в словаре вернуть `<UNK>`)

Первые 4 индекса зарезервированы под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`.
    
Создайте `Vocab` на основе списка `corpus` и закодируйте каждый токен в предложениях, используя `Vocab`. Выведите полученный результат на экран.

- [x] Проверено на семинаре

In [19]:
class Vocab:
    def __init__(self, data):
        self.words = set()
        for tokens in data:
          for tok in tokens:
              if tok not in ['<pad>', '<unk>', '<sos>', '<eos>']:
                  self.words.update([tok])
        self.words = ['<pad>', '<unk>', '<sos>', '<eos>'] + list(self.words)
        self.dct = {v: idx for idx, v in enumerate(self.words)}

    def itos(self, idx):
        """Возвращает токен по индексу"""
        try:
          return self.words[idx]
        except IndexError:
          return '<UNK>'

    def stoi(self, s):
        """Возвращает индекс токена"""
        return self.dct.get(s.lower(), 1)

In [20]:
corpus = [
    "Маленький котенок игриво прыгает за шариком",
    "Пушистый котик мурлыкает, лежа на солнышке",
    "Котенок любопытно нюхает цветы в саду",
    "Котик ловко лазает по дереву, исследуя окружающий мир",
    "Спящий котик мило моргает своими яркими глазками"
]

In [21]:
import re

In [22]:
corpus_t = [
    word_tokenize(re.sub('\W+', ' ', text).lower())
    for text in corpus
]

In [23]:
corpus_t

[['маленький', 'котенок', 'игриво', 'прыгает', 'за', 'шариком'],
 ['пушистый', 'котик', 'мурлыкает', 'лежа', 'на', 'солнышке'],
 ['котенок', 'любопытно', 'нюхает', 'цветы', 'в', 'саду'],
 ['котик', 'ловко', 'лазает', 'по', 'дереву', 'исследуя', 'окружающий', 'мир'],
 ['спящий', 'котик', 'мило', 'моргает', 'своими', 'яркими', 'глазками']]

In [24]:
v = Vocab(corpus_t)

In [25]:
v.itos(100)

'<UNK>'

In [26]:
v.stoi('яркими')

24

In [27]:
v.stoi('трубка')

1

In [28]:
v.stoi('<PAD>')

0

In [29]:
vectors = [[v.stoi(word) for word in sent] for sent in corpus_t]

In [30]:
vectors

[[33, 32, 26, 12, 7, 22],
 [5, 11, 20, 27, 28, 21],
 [32, 8, 6, 18, 14, 25],
 [11, 16, 10, 4, 30, 31, 9, 17],
 [13, 11, 29, 19, 23, 24, 15]]

<p class="task" id="2"></p>

2\. Создайте класс `NewsDataset` на основе данных из файла `news.csv`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости (или новостей, если используются срезы) и метки классов для этих новостей. Для кодирования текстов используйте собственную реализацию `Vocab`. Там, где это возможно, возвращайте результат в виде тензора, а не списка.

Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [31]:
from torch.utils.data import Dataset
import re
from nltk.tokenize import RegexpTokenizer

In [32]:
class NewsDataset(Dataset):
    def __init__(self, data):
      self.text = data['text'].values
      self.label = data['label'].values
      tokenizer = RegexpTokenizer(r'(<pad>|<unk>|<sos>|<eos>|\w+)')
      self.corpus = [
          tokenizer.tokenize(text.lower())
          for text in self.text
      ]
      self.vocab = Vocab(self.corpus)

    def __getitem__(self, idx):
      t = self.corpus[idx]
      l = self.label[idx]
      if type(t[0]) == str:
          vectors = [self.vocab.stoi(word) for word in t]
      else:
          vectors = [[self.vocab.stoi(word) for word in sent] for sent in t]
      try:
          return th.tensor(vectors), l
      except ValueError:
          return vectors, l

In [33]:
import pandas as pd

df = pd.read_csv('news.csv')
df.head()

,text,label
0,лукашенко пригрозил литовским танкам белорусск...,0
1,российские компании оказались в опасности из з...,1
2,лукашенко объявил об отмене выборов из за напа...,0
3,«роснефть» стала лидером по объему биржевых пр...,1
4,псковской области дадут десятки миллионов рубл...,1


In [34]:
dataset = NewsDataset(df)

In [35]:
dataset[0]

(tensor([2929, 2663, 5686,  553,  704,  629]), 0)

In [36]:
dataset[:3]

([[2929, 2663, 5686, 553, 704, 629],
  [118, 5899, 3778, 5128, 2665, 4686, 5066, 1383, 3437],
  [2929, 3546, 1915, 1679, 3307, 4686, 5066, 4801, 5161, 4544]],
 array([0, 1, 0]))

<p class="task" id="3"></p>

3\. Реализуйте преобразование `Truncate`, которое обрезает каждый текст в батче до `n` символов. Создайте версию `NewsDataset` с обрезкой предложений до 5 токенов. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [37]:
class Truncate:
    def __init__(self, n, data):
        self.n = n
        self.data = data.copy()
    def cut(self):
        self.data['text'] = self.data['text'].map(lambda x: ' '.join(x.split()[:self.n]))
        return self.data

In [38]:
t = Truncate(5, df)

In [39]:
truncated_dataset = NewsDataset(t.cut())

In [40]:
truncated_dataset[0]

(tensor([3117, 2584, 2563, 1074, 1392]), 0)

In [41]:
truncated_dataset[:5]

(tensor([[3117, 2584, 2563, 1074, 1392],
         [ 225, 2968, 1754, 1384, 2589],
         [3117, 1278, 1001,  515,  779],
         [2737,  312, 2781, 1116,  444],
         [1797, 2988, 2375,  543, 2674]]),
 array([0, 1, 0, 1, 1]))

<p class="task" id="4"></p>

4\. Реализуйте преобразование `Pad`, которое расширяет каждый текст в батче до `n` символов значением `pad_idx`. Создайте версию `NewsDataset` с расширением предложений до 30 символов. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [42]:
class Pad:
    def __init__(self, n, pad_idx, data):
        self.n = n
        self.pad_idx = pad_idx
        self.data = data.copy()

    def extend(self):
        self.data['text'] = self.data['text'].map(
            lambda x: x + ' ' + (self.pad_idx + ' ') * (self.n - len(x.split()))
            )
        return self.data

In [43]:
P = Pad(30, '<PAD>', df)

In [44]:
padded_dataset = NewsDataset(P.extend())

In [45]:
padded_dataset[0]

(tensor([2929, 2663, 5686,  553,  704,  629,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
 0)

In [46]:
padded_dataset[:3]

(tensor([[2929, 2663, 5686,  553,  704,  629,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0],
         [ 118, 5899, 3778, 5128, 2665, 4686, 5066, 1383, 3437,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0],
         [2929, 3546, 1915, 1679, 3307, 4686, 5066, 4801, 5161, 4544,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0]]),
 array([0, 1, 0]))

<p class="task" id="5"></p>

5\. Создайте объект `torchtext.vocab.Vocab` на основе данных из файла `news.csv`. Первые 4 индекса зарезервируйте под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`. Опишите класс `NewsDatasetTorchText`, аналогичный по функционалу классу `NewsDataset`, но использующего реализацию `torchtext.vocab.Vocab`. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [47]:
import torchtext

In [48]:
from torchtext.vocab import build_vocab_from_iterator

In [49]:
corpus = [
              word_tokenize(re.sub(r'[^\w\s<>]', ' ', text.lower()))
              for text in df['text']
          ]

In [50]:
vocab = build_vocab_from_iterator(corpus, specials=["<pad>", "<unk>", "sos", "eos"])

In [69]:
class NewsDatasetTorchText(Dataset):
    def __init__(self, data, vocab):
      self.text = data['text'].values
      self.label = data['label'].values
      tokenizer = RegexpTokenizer(r'(<pad>|<unk>|<sos>|<eos>|\w+)')
      self.corpus = [
          tokenizer.tokenize(text.lower())
          for text in self.text
      ]
      self.vocab = vocab

    def __getitem__(self, idx):
      t = self.corpus[idx]
      l = self.label[idx]
      if type(t[0]) == str:
          vectors = [self.vocab.get_stoi()[word] for word in t]
      else:
          vectors = [[self.vocab.get_stoi()[word] for word in sent] for sent in t]
      try:
          return th.tensor(vectors), l
      except ValueError:
          return vectors, l
      return vectors

In [71]:
dataset_t = NewsDatasetTorchText(df, vocab)

In [72]:
dataset_t[0]

(tensor([  28,  198, 3349, 5408,  486, 3089]), 0)

In [73]:
dataset_t[:3]

([[28, 198, 3349, 5408, 486, 3089],
  [24, 144, 3907, 4, 374, 10, 8, 2418, 4479],
  [28, 61, 26, 4030, 313, 10, 8, 3661, 563, 988]],
 array([0, 1, 0]))

<p class="task" id="6"></p>

6\. Создайте преобразование, которое последовательно:
* преобразует набор токенов в последовательность индексов;
* преобразует результат в тензор;
* расширяет предложения до 30 символов, заполняя недостающие позиции индексом 0.

Создайте версию NewsDatasetTorchText с указанием этого преобразования. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [75]:
class NewsDatasetTorchTextTransformed(Dataset):
    def __init__(self, data, vocab, transform):
      if transform:
        pad = Pad(30, '<PAD>', data)
        data = pad.extend()
      self.text = data['text'].values
      self.label = data['label'].values
      self.transform = transform
      tokenizer = RegexpTokenizer(r'(<pad>|<unk>|<sos>|<eos>|\w+)')
      self.corpus = [
          tokenizer.tokenize(text.lower())
          for text in self.text
      ]
      self.vocab = vocab

    def __getitem__(self, idx):
      t = self.corpus[idx]
      l = self.label[idx]
      if type(t[0]) == str:
          vectors = [self.vocab.get_stoi()[word] for word in t]
      else:
          vectors = [[self.vocab.get_stoi()[word] for word in sent] for sent in t]
      try:
          return th.tensor(vectors), l
      except ValueError:
          return vectors, l
      return vectors

In [76]:
ndt = NewsDatasetTorchTextTransformed(df, vocab, transform=True)

In [77]:
ndt[0]

(tensor([  28,  198, 3349, 5408,  486, 3089,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
 0)

In [78]:
ndt[:3]

(tensor([[  28,  198, 3349, 5408,  486, 3089,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0],
         [  24,  144, 3907,    4,  374,   10,    8, 2418, 4479,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0],
         [  28,   61,   26, 4030,  313,   10,    8, 3661,  563,  988,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0]]),
 array([0, 1, 0]))

## Обратная связь
- [x] Хочу получить обратную связь по решению